<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Experimento 2 AutoGluon</h4>
</center>

In [1]:
# !pip install autogluon
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [2]:
df = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product.csv", header = None)
df.columns = ["timestamp", "item_id", "target"]
df.head(5)

,timestamp,item_id,target
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011


In [3]:
data = TimeSeriesDataFrame(df)
data

,,target
item_id,timestamp,
20001,2017-01-01,934.77222
20002,2017-01-01,550.15707
20003,2017-01-01,1063.45835
20004,2017-01-01,555.91614
20005,2017-01-01,494.27011
...,...,...
21263,2019-12-01,0.01270
21265,2019-12-01,0.05007
21266,2019-12-01,0.05121


In [8]:
from autogluon.timeseries.metrics import TimeSeriesScorer

class CustomWeightedErrorMetric(TimeSeriesScorer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def compute_metric(self, data_future, predictions, target, **kwargs):
        total_weighted_error = 0.0
        
        # Obtener todas las ventas totales de todos los productos
        total_actual_sales_all_products = np.sum(data_future[target])
        
        # Iterar sobre cada índice (item_id) en el TimeSeriesDataFrame
        for item_id in data_future.index:
            # Filtrar los datos para el item_id actual
            product_y_true = data_future.loc[item_id, target]
            product_y_pred = predictions.loc[item_id, "mean"]
            
            # Calcular el peso del item_id actual basado en las ventas totales de todos los productos
            product_total_actual_sales = np.sum(product_y_true)
            product_weight = product_total_actual_sales / total_actual_sales_all_products
            
            # Calcular el error ponderado para el item_id actual
            product_absolute_errors = np.abs(product_y_true - product_y_pred)
            product_weighted_errors = product_absolute_errors * product_weight
            product_weighted_error = np.sum(product_weighted_errors)
            
            # Sumar al error total ponderado
            total_weighted_error += product_weighted_error
        
        return total_weighted_error / len(data_future.index) 

In [9]:
predictor = TimeSeriesPredictor(target='target', prediction_length=2, freq="M", eval_metric=CustomWeightedErrorMetric()).fit(data, num_val_windows = 3)
predictions = predictor.predict(data)

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20240702_222840'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       2.21 GB / 7.88 GB (28.0%)
Disk Space Avail:   6.41 GB / 110.21 GB (5.8%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': CustomWeightedErrorMetric,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 3,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

train_data with frequenc

In [10]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1213.985301
3,20002,1014.150400
5,20003,686.287721
7,20004,497.548223
9,20005,497.443168


In [11]:
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_3ventanas_v3.csv", sep = ",", index = False)